In [2]:
import pytz
from datetime import datetime, timedelta
from gym_mtsim import MtSimulator, OrderType, Timeframe, FOREX_DATA_PATH
import matplotlib.pyplot as plt

selected_symbol = ['EURUSD', 'GBPCAD', 'GBPUSD', 'USDCAD', 'USDCHF', 'GBPJPY', 'USDJPY']
file_name = '1D_test_final.pkl'

# Create sim
sim = MtSimulator(
    unit='USD',
    balance=10000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=False,
)

# Load or create new file
if not sim.load_symbols(file_name):
    sim.download_data(
        symbols = selected_symbol,
        time_range=(
            datetime(2021, 1, 1, tzinfo=pytz.UTC),
            datetime(2021, 12, 31, tzinfo=pytz.UTC)
        ),
        timeframe=Timeframe.D1
        #timeframe=Timeframe.D1
    )

# Define df

In [3]:
df = sim.symbols_data

# Change column name to lower case for finTA

In [4]:
for i in selected_symbol:
    df[i] = df[i].rename(columns=str.lower)

df

{'EURUSD':                               open    close      low     high  volume
 Time                                                                 
 2021-01-04 00:00:00+00:00  1.22395  1.22473  1.22280  1.23098   85620
 2021-01-05 00:00:00+00:00  1.22466  1.22965  1.22432  1.23056   75380
 2021-01-06 00:00:00+00:00  1.22968  1.23252  1.22657  1.23494  118259
 2021-01-07 00:00:00+00:00  1.23256  1.22710  1.22452  1.23441   84749
 2021-01-08 00:00:00+00:00  1.22713  1.22218  1.21932  1.22845  100138
 ...                            ...      ...      ...      ...     ...
 2021-12-27 00:00:00+00:00  1.13023  1.13264  1.13023  1.13342   37691
 2021-12-28 00:00:00+00:00  1.13260  1.13079  1.12894  1.13330   40037
 2021-12-29 00:00:00+00:00  1.13079  1.13477  1.12732  1.13687   51231
 2021-12-30 00:00:00+00:00  1.13445  1.13256  1.12988  1.13599   50314
 2021-12-31 00:00:00+00:00  1.13244  1.13755  1.13033  1.13861   39263
 
 [260 rows x 5 columns],
 'GBPCAD':                              

# Change volume data type to float

In [5]:
for symbol in range(0,len(selected_symbol)):
    #print(type(sim.symbols_data[selected_symbol[symbol]]))
    df[selected_symbol[symbol]]['volume'] = df[selected_symbol[symbol]]['volume'].astype(float)
print(df[selected_symbol[2]].dtypes)

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object


## Add indicator for each selected_symbol

In [6]:
from finta import TA


for symbol in range(0,len(selected_symbol)):
    df[selected_symbol[symbol]]['EFI'] = TA.EFI(df[selected_symbol[symbol]])
    df[selected_symbol[symbol]]['ATR'] = TA.ATR(df[selected_symbol[symbol]])
    df[selected_symbol[symbol]]['ADX'] = TA.ADX(df[selected_symbol[symbol]])
    df[selected_symbol[symbol]]['STOCHRSI'] = TA.STOCHRSI(df[selected_symbol[symbol]])

df[selected_symbol[0]].head()

,open,close,low,high,volume,EFI,ATR,ADX,STOCHRSI
Time,,,,,,,,,
2021-01-04 00:00:00+00:00,1.22395,1.22473,1.22280,1.23098,85620.0,NaN,NaN,NaN,NaN
2021-01-05 00:00:00+00:00,1.22466,1.22965,1.22432,1.23056,75380.0,370.869600,NaN,NaN,NaN
2021-01-06 00:00:00+00:00,1.22968,1.23252,1.22657,1.23494,118259.0,353.926224,NaN,NaN,NaN
2021-01-07 00:00:00+00:00,1.23256,1.22710,1.22452,1.23441,84749.0,40.146504,NaN,NaN,NaN
2021-01-08 00:00:00+00:00,1.22713,1.22218,1.21932,1.22845,100138.0,-125.246377,NaN,NaN,NaN


In [7]:
# Fill NA

for symbol in range(0,len(selected_symbol)):
    df[selected_symbol[symbol]].fillna(0, inplace=True)

df[selected_symbol[0]].head()

,open,close,low,high,volume,EFI,ATR,ADX,STOCHRSI
Time,,,,,,,,,
2021-01-04 00:00:00+00:00,1.22395,1.22473,1.22280,1.23098,85620.0,0.000000,0.0,0.0,0.0
2021-01-05 00:00:00+00:00,1.22466,1.22965,1.22432,1.23056,75380.0,370.869600,0.0,0.0,0.0
2021-01-06 00:00:00+00:00,1.22968,1.23252,1.22657,1.23494,118259.0,353.926224,0.0,0.0,0.0
2021-01-07 00:00:00+00:00,1.23256,1.22710,1.22452,1.23441,84749.0,40.146504,0.0,0.0,0.0
2021-01-08 00:00:00+00:00,1.22713,1.22218,1.21932,1.22845,100138.0,-125.246377,0.0,0.0,0.0


In [10]:
indicator_list = ['EFI','ATR','ADX','STOCHRSI']

for symbol in range(0,len(selected_symbol)):
    print('------')
    print('Corr for', selected_symbol[symbol])
    print('------')
    print(df[selected_symbol[symbol]][indicator_list].corr())

------
Corr for EURUSD
------
               EFI       ATR       ADX  STOCHRSI
EFI       1.000000 -0.107502  0.031444  0.213491
ATR      -0.107502  1.000000  0.548072  0.363950
ADX       0.031444  0.548072  1.000000  0.035996
STOCHRSI  0.213491  0.363950  0.035996  1.000000
------
Corr for GBPCAD
------
               EFI       ATR       ADX  STOCHRSI
EFI       1.000000  0.144158  0.173415  0.375082
ATR       0.144158  1.000000  0.294900  0.676054
ADX       0.173415  0.294900  1.000000  0.322183
STOCHRSI  0.375082  0.676054  0.322183  1.000000
------
Corr for GBPUSD
------
               EFI       ATR       ADX  STOCHRSI
EFI       1.000000 -0.213401  0.006474  0.148272
ATR      -0.213401  1.000000  0.467618  0.471424
ADX       0.006474  0.467618  1.000000  0.075965
STOCHRSI  0.148272  0.471424  0.075965  1.000000
------
Corr for USDCAD
------
               EFI       ATR       ADX  STOCHRSI
EFI       1.000000  0.274399  0.265764  0.445274
ATR       0.274399  1.000000  0.532669  0.77366

In [11]:
for symbol in range(0,len(selected_symbol)):
    df[selected_symbol[symbol]] = df[selected_symbol[symbol]][['open', 'close', 'low', 'high', 'volume'] + indicator_list]
    print(df[selected_symbol[symbol]].head())

                              open    close      low     high    volume  \
Time                                                                      
2021-01-04 00:00:00+00:00  1.22395  1.22473  1.22280  1.23098   85620.0   
2021-01-05 00:00:00+00:00  1.22466  1.22965  1.22432  1.23056   75380.0   
2021-01-06 00:00:00+00:00  1.22968  1.23252  1.22657  1.23494  118259.0   
2021-01-07 00:00:00+00:00  1.23256  1.22710  1.22452  1.23441   84749.0   
2021-01-08 00:00:00+00:00  1.22713  1.22218  1.21932  1.22845  100138.0   

                                  EFI  ATR  ADX  STOCHRSI  
Time                                                       
2021-01-04 00:00:00+00:00    0.000000  0.0  0.0       0.0  
2021-01-05 00:00:00+00:00  370.869600  0.0  0.0       0.0  
2021-01-06 00:00:00+00:00  353.926224  0.0  0.0       0.0  
2021-01-07 00:00:00+00:00   40.146504  0.0  0.0       0.0  
2021-01-08 00:00:00+00:00 -125.246377  0.0  0.0       0.0  
                              open    close      low  

In [13]:
df[selected_symbol[0]].shape

(260, 9)

In [14]:
df

{'EURUSD':                               open    close      low     high    volume  \
 Time                                                                      
 2021-01-04 00:00:00+00:00  1.22395  1.22473  1.22280  1.23098   85620.0   
 2021-01-05 00:00:00+00:00  1.22466  1.22965  1.22432  1.23056   75380.0   
 2021-01-06 00:00:00+00:00  1.22968  1.23252  1.22657  1.23494  118259.0   
 2021-01-07 00:00:00+00:00  1.23256  1.22710  1.22452  1.23441   84749.0   
 2021-01-08 00:00:00+00:00  1.22713  1.22218  1.21932  1.22845  100138.0   
 ...                            ...      ...      ...      ...       ...   
 2021-12-27 00:00:00+00:00  1.13023  1.13264  1.13023  1.13342   37691.0   
 2021-12-28 00:00:00+00:00  1.13260  1.13079  1.12894  1.13330   40037.0   
 2021-12-29 00:00:00+00:00  1.13079  1.13477  1.12732  1.13687   51231.0   
 2021-12-30 00:00:00+00:00  1.13445  1.13256  1.12988  1.13599   50314.0   
 2021-12-31 00:00:00+00:00  1.13244  1.13755  1.13033  1.13861   39263.0   
 


# Change column name to match with observation space

In [15]:
# Change column to lower case
for i in selected_symbol:
    #sim.symbols_data[i] = sim.symbols_data[i].rename({'close':'Close'})
    df[i].rename({'close':'Close', 'open':'Open', 'low':'Low','high':'High','volume':'Volume'}, axis=1, inplace=True)

df

{'EURUSD':                               Open    Close      Low     High    Volume  \
 Time                                                                      
 2021-01-04 00:00:00+00:00  1.22395  1.22473  1.22280  1.23098   85620.0   
 2021-01-05 00:00:00+00:00  1.22466  1.22965  1.22432  1.23056   75380.0   
 2021-01-06 00:00:00+00:00  1.22968  1.23252  1.22657  1.23494  118259.0   
 2021-01-07 00:00:00+00:00  1.23256  1.22710  1.22452  1.23441   84749.0   
 2021-01-08 00:00:00+00:00  1.22713  1.22218  1.21932  1.22845  100138.0   
 ...                            ...      ...      ...      ...       ...   
 2021-12-27 00:00:00+00:00  1.13023  1.13264  1.13023  1.13342   37691.0   
 2021-12-28 00:00:00+00:00  1.13260  1.13079  1.12894  1.13330   40037.0   
 2021-12-29 00:00:00+00:00  1.13079  1.13477  1.12732  1.13687   51231.0   
 2021-12-30 00:00:00+00:00  1.13445  1.13256  1.12988  1.13599   50314.0   
 2021-12-31 00:00:00+00:00  1.13244  1.13755  1.13033  1.13861   39263.0   
 


# Check suitable window size

In [16]:
df['EURUSD'].head()

,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI
Time,,,,,,,,,
2021-01-04 00:00:00+00:00,1.22395,1.22473,1.22280,1.23098,85620.0,0.000000,0.0,0.0,0.0
2021-01-05 00:00:00+00:00,1.22466,1.22965,1.22432,1.23056,75380.0,370.869600,0.0,0.0,0.0
2021-01-06 00:00:00+00:00,1.22968,1.23252,1.22657,1.23494,118259.0,353.926224,0.0,0.0,0.0
2021-01-07 00:00:00+00:00,1.23256,1.22710,1.22452,1.23441,84749.0,40.146504,0.0,0.0,0.0
2021-01-08 00:00:00+00:00,1.22713,1.22218,1.21932,1.22845,100138.0,-125.246377,0.0,0.0,0.0


In [22]:
# start => window size should start at 15 to prevent 0 data
start = 13
end = 20

for i in selected_symbol:
    print("--------")
    print(i)
    print("--------")
    print(df[i][['ADX','STOCHRSI','ATR']].iloc[start-1:end])

--------
EURUSD
--------
                                  ADX  STOCHRSI       ATR
Time                                                     
2017-01-18 00:00:00+00:00    0.000000  0.000000  0.000000
2017-01-19 00:00:00+00:00  100.000000  0.000000  0.010334
2017-01-20 00:00:00+00:00   50.876194  0.746335  0.010368
2017-01-23 00:00:00+00:00   49.458822  0.806449  0.009839
2017-01-24 00:00:00+00:00   49.496272  0.806545  0.009442
2017-01-25 00:00:00+00:00   46.644795  0.792490  0.008899
2017-01-26 00:00:00+00:00   39.431796  0.790846  0.008977
2017-01-27 00:00:00+00:00   34.307776  0.785201  0.008939
--------
GBPCAD
--------
                                  ADX  STOCHRSI       ATR
Time                                                     
2017-01-18 00:00:00+00:00    0.000000  0.000000  0.000000
2017-01-19 00:00:00+00:00  100.000000  0.000000  0.018488
2017-01-20 00:00:00+00:00  100.000000  0.265345  0.018316
2017-01-23 00:00:00+00:00  100.000000  0.320730  0.018394
2017-01-24 00:00:00+00

In [19]:
# Create a copy of open, close

for i in selected_symbol:
    df[i]['open'] = df[i]['Open']
    df[i]['close'] = df[i]['Close']
    df[i]['low'] = df[i]['Low']
    df[i]['high'] = df[i]['High']

df['EURUSD'].head()

,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2021-01-04 00:00:00+00:00,1.22395,1.22473,1.22280,1.23098,85620.0,0.000000,0.0,0.0,0.0,1.22395,1.22473,1.22280,1.23098
2021-01-05 00:00:00+00:00,1.22466,1.22965,1.22432,1.23056,75380.0,370.869600,0.0,0.0,0.0,1.22466,1.22965,1.22432,1.23056
2021-01-06 00:00:00+00:00,1.22968,1.23252,1.22657,1.23494,118259.0,353.926224,0.0,0.0,0.0,1.22968,1.23252,1.22657,1.23494
2021-01-07 00:00:00+00:00,1.23256,1.22710,1.22452,1.23441,84749.0,40.146504,0.0,0.0,0.0,1.23256,1.22710,1.22452,1.23441
2021-01-08 00:00:00+00:00,1.22713,1.22218,1.21932,1.22845,100138.0,-125.246377,0.0,0.0,0.0,1.22713,1.22218,1.21932,1.22845


In [20]:
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np

# Volume
#columns = ['Open','Close','Low','High','Volume','EFI','ADX','ATR','STOCHRSI']
columns = ['open','close','low','high','Volume','EFI','ADX','ATR','STOCHRSI']


for i in selected_symbol:
    scaler_filename = "scaler/scaler_" + i + "_1D"
    print("Load scaler from",scaler_filename)
    scaler = joblib.load(scaler_filename) 
    df[i][columns] = scaler.transform(df[i][columns])
    display(df[i])


Load scaler from scaler/scaler_EURUSD_1D


,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2021-01-04 00:00:00+00:00,1.22395,1.22473,1.22280,1.23098,0.251820,0.584284,0.000000,0.000000,0.000000,0.872652,0.876153,0.895641,0.881100
2021-01-05 00:00:00+00:00,1.22466,1.22965,1.22432,1.23056,0.217480,0.742928,0.000000,0.000000,0.000000,0.876028,0.899544,0.902851,0.879067
2021-01-06 00:00:00+00:00,1.22968,1.23252,1.22657,1.23494,0.361278,0.735680,0.000000,0.000000,0.000000,0.899900,0.913188,0.913524,0.900271
2021-01-07 00:00:00+00:00,1.23256,1.22710,1.22452,1.23441,0.248899,0.601457,0.000000,0.000000,0.000000,0.913596,0.887420,0.903800,0.897705
2021-01-08 00:00:00+00:00,1.22713,1.22218,1.21932,1.22845,0.300507,0.530708,0.000000,0.000000,0.000000,0.887774,0.864030,0.879133,0.868852
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 00:00:00+00:00,1.13023,1.13264,1.13023,1.13342,0.091087,0.594212,0.334772,0.146686,0.300534,0.426982,0.438338,0.456525,0.408792
2021-12-28 00:00:00+00:00,1.13260,1.13079,1.12894,1.13330,0.098954,0.588267,0.317503,0.140789,0.301284,0.438252,0.429543,0.450406,0.408211
2021-12-29 00:00:00+00:00,1.13079,1.13477,1.12732,1.13687,0.136494,0.600158,0.327937,0.135760,0.311636,0.429645,0.448464,0.442721,0.425494


Load scaler from scaler/scaler_GBPCAD_1D


,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2021-01-04 00:00:00+00:00,1.73554,1.73315,1.72462,1.74121,0.302972,0.486287,0.000000,0.000000,0.000000,0.619143,0.591844,0.595516,0.603148
2021-01-05 00:00:00+00:00,1.73346,1.72598,1.72475,1.73617,0.324517,0.265724,0.000000,0.000000,0.000000,0.611141,0.562981,0.596028,0.583218
2021-01-06 00:00:00+00:00,1.72603,1.72455,1.71955,1.73129,0.412500,0.354380,0.000000,0.000000,0.000000,0.582558,0.557224,0.575538,0.563921
2021-01-07 00:00:00+00:00,1.72441,1.72088,1.71880,1.73033,0.316711,0.362763,0.000000,0.000000,0.000000,0.576325,0.542450,0.572583,0.560125
2021-01-08 00:00:00+00:00,1.72088,1.72122,1.71881,1.72724,0.337404,0.404481,0.000000,0.000000,0.000000,0.562745,0.543819,0.572622,0.547906
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 00:00:00+00:00,1.71307,1.71861,1.71307,1.72336,0.099363,0.524035,0.263481,0.194892,0.804950,0.532700,0.533312,0.550004,0.532564
2021-12-28 00:00:00+00:00,1.71835,1.72116,1.71646,1.72310,0.103647,0.522222,0.252473,0.201701,0.835831,0.553012,0.543577,0.563362,0.531535
2021-12-29 00:00:00+00:00,1.71910,1.72498,1.71910,1.72841,0.135603,0.524103,0.247460,0.213179,0.861897,0.555898,0.558955,0.573765,0.552533


Load scaler from scaler/scaler_GBPUSD_1D


...

,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2021-01-04 00:00:00+00:00,0.88219,0.88121,0.87847,0.88457,0.264284,0.486157,0.000000,0.000000,0.000000,0.007444,0.000274,-0.006469,-0.005741
2021-01-05 00:00:00+00:00,0.88117,0.87826,0.87757,0.88167,0.266096,0.374533,0.000000,0.000000,0.000000,0.000478,-0.019955,-0.012867,-0.025328
2021-01-06 00:00:00+00:00,0.87825,0.87827,0.87577,0.88216,0.410166,0.434938,0.000000,0.000000,0.000000,-0.019465,-0.019886,-0.025663,-0.022018
2021-01-07 00:00:00+00:00,0.87796,0.88492,0.87746,0.88607,0.327079,0.571162,0.000000,0.000000,0.000000,-0.021445,0.025715,-0.013649,0.004390
2021-01-08 00:00:00+00:00,0.88492,0.88531,0.88227,0.88848,0.362721,0.550805,0.000000,0.000000,0.000000,0.026089,0.028389,0.020545,0.020667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 00:00:00+00:00,0.91743,0.91749,0.91711,0.91999,0.050633,0.473242,0.354503,0.101848,0.758973,0.248122,0.249057,0.268216,0.233486
2021-12-28 00:00:00+00:00,0.91743,0.91735,0.91592,0.91904,0.063338,0.474833,0.342984,0.106233,0.755270,0.248122,0.248097,0.259757,0.227070
2021-12-29 00:00:00+00:00,0.91712,0.91457,0.91269,0.92014,0.086944,0.470234,0.341606,0.119014,0.743151,0.246005,0.229034,0.236795,0.234500


Load scaler from scaler/scaler_GBPJPY_1D


,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2021-01-04 00:00:00+00:00,140.657,139.903,139.689,141.307,0.132005,0.657652,0.000000,0.000000,0.000000,0.503387,0.477591,0.498853,0.474156
2021-01-05 00:00:00+00:00,139.902,139.964,139.509,140.152,0.129067,0.671378,0.000000,0.000000,0.000000,0.478913,0.479574,0.493118,0.434463
2021-01-06 00:00:00+00:00,139.963,140.217,139.746,140.578,0.180045,0.703136,0.000000,0.000000,0.000000,0.480891,0.487796,0.500669,0.449103
2021-01-07 00:00:00+00:00,140.196,140.751,140.056,140.998,0.132525,0.732820,0.000000,0.000000,0.000000,0.488444,0.505151,0.510546,0.463537
2021-01-08 00:00:00+00:00,140.755,140.989,140.650,141.373,0.141612,0.727245,0.000000,0.000000,0.000000,0.506564,0.512886,0.529472,0.476424
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 00:00:00+00:00,152.928,154.370,152.928,154.454,0.036171,0.700763,0.303073,0.256411,0.285185,0.901164,0.947772,0.920665,0.925974
2021-12-28 00:00:00+00:00,154.370,154.211,153.998,154.572,0.039573,0.691979,0.288464,0.260767,0.313255,0.947908,0.942605,0.954757,0.930030
2021-12-29 00:00:00+00:00,154.165,155.047,154.045,155.128,0.054746,0.703094,0.293253,0.268511,0.345644,0.941262,0.969775,0.956254,0.949137


Load scaler from scaler/scaler_USDJPY_1D


,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2021-01-04 00:00:00+00:00,103.121,103.126,102.712,103.316,0.141530,0.465987,0.000000,0.000000,0.000000,0.055570,0.055854,0.095740,0.000000
2021-01-05 00:00:00+00:00,103.100,102.711,102.605,103.191,0.134994,0.391077,0.000000,0.000000,0.000000,0.054211,0.029026,0.089066,-0.008177
2021-01-06 00:00:00+00:00,102.701,103.036,102.591,103.444,0.204939,0.476250,0.000000,0.000000,0.000000,0.028400,0.050036,0.088193,0.008374
2021-01-07 00:00:00+00:00,103.032,103.795,102.951,103.957,0.165115,0.534697,0.000000,0.000000,0.000000,0.049812,0.099101,0.110647,0.041934
2021-01-08 00:00:00+00:00,103.796,103.967,103.604,104.092,0.184510,0.525868,0.000000,0.000000,0.000000,0.099237,0.110220,0.151375,0.050765
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 00:00:00+00:00,114.304,114.885,114.304,114.913,0.066249,0.478736,0.214821,0.165084,1.025910,0.779014,0.816019,0.818749,0.758668
2021-12-28 00:00:00+00:00,114.878,114.819,114.705,114.948,0.076795,0.475807,0.203352,0.178417,1.042322,0.816147,0.811753,0.843760,0.760958
2021-12-29 00:00:00+00:00,114.800,114.960,114.664,115.038,0.095365,0.477176,0.198633,0.192786,1.063638,0.811101,0.820868,0.841203,0.766845


In [21]:
#Save data
sim.save_symbols(file_name)